In [1]:
import math
import pandas as pd
import re
import numpy as np
import itertools
import collections
from collections import defaultdict, Counter
import string
from sklearn.preprocessing import LabelEncoder

In [2]:
ls

 Volume in drive C is Acer
 Volume Serial Number is 3818-7542

 Directory of C:\Users\phvpa\Desktop\Experimants\DataSet

30-10-2021  15:23    <DIR>          .
30-10-2021  12:45    <DIR>          ..
30-10-2021  15:16    <DIR>          .ipynb_checkpoints
30-10-2021  15:23            36,012 Kan_Preprocessing _and_spliting_2750and250.ipynb
30-10-2021  12:45    <DIR>          Kannada
30-10-2021  13:57            38,450 Mal_Preprocessing _and_spliting_5000and500.ipynb
30-10-2021  12:45    <DIR>          Malyalam
30-10-2021  12:54    <DIR>          NumpyFiles
30-10-2021  12:45    <DIR>          Tamil
               2 File(s)         74,462 bytes
               7 Dir(s)  311,477,403,648 bytes free


In [3]:
#pd.options.display.max_rows = None

In [4]:
#"C:\Users\phvpa\Desktop\Experimants\DataSet\Kannada\kannada_sentiment_full_train.tsv"
#"C:\Users\phvpa\Desktop\Experimants\DataSet\Kannada\kannada_sentiment_full_test_withoutlabels.tsv"
#"C:\Users\phvpa\Desktop\Experimants\DataSet\Kannada\kannada_sentiment_full_dev.tsv"
train_df = pd.read_csv("Kannada\kannada_sentiment_full_train.tsv", sep='\t')
print(len(train_df))

#test_df = pd.read_csv("Mal_sentiment_full_dev.tsv", header=None, sep='\t')
valid_df = pd.read_csv("Kannada\kannada_sentiment_full_dev.tsv", sep='\t')
print(len(valid_df))

test_df = pd.read_csv("Kannada\kannada_sentiment_full_test_withoutlabels.tsv", sep='\t')
len(test_df)

6212
691


768

In [5]:
train_df

,text,category
0,ಒಂದು ದೇಶದ ಮುಂದುವರಿಯುವುದು ಅದರ ಆರ್ಥಿಕ ಸ್ಥಿತಿಯನ್ನ...,Negative
1,ಕನ್ನಡದಲ್ಲಿ ಡೈಲಿ ಟೆಕ್ ಅಪ್ಡೇಟ್ಸ್ ಪಡೆಯಲು ಸಬ್ಸ್ಕ್ರ...,Positive
2,Super sar song,not-Kannada
3,Tiktokers present situation... nನೋಡುವವರು ಯಾರು ...,Negative
4,Super ಸಾಂಗ್ ವೆರಿ ನೈಸ್....,Positive
...,...,...
6207,@A.R.W tumbad tanhaji andhadhun aise bahot h...,not-Kannada
6208,പൊളി ഡാൻസ്🥰 രക്ഷിത് ഷെട്ടി മാസ്സ്,not-Kannada
6209,Bro...nNeen este Roast madudru...China ne beku...,Negative
6210,ಕೌಶಲ್ಯ ಇದ್ದವರ ಸಂಖ್ಯೆ ಕಡಿಮೆ ಇಲ್ಲ ಸರ್ ತುಂಬಾ ಪ್ರತ...,Positive


In [6]:
train_df['category']

0             Negative
1             Positive
2          not-Kannada
3             Negative
4             Positive
             ...      
6207       not-Kannada
6208       not-Kannada
6209          Negative
6210          Positive
6211    Mixed feelings
Name: category, Length: 6212, dtype: object

In [7]:
#removing all the emoji from the given data

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)


for i in range(len(train_df)):
    temp1 = deEmojify(train_df['text'][i])
    train_df['text'][i] = temp1

for j in range(len(valid_df)):
    temp2 = deEmojify(valid_df['text'][j])
    valid_df['text'][j] = temp2

for k in range(len(test_df)):
    temp3 = deEmojify(test_df['text'][k])
    test_df['text'][k]=temp3

In [8]:
def preprocess_data(text):
  user_name = '@[A-Za-z0-9]+' # Pattern for matching the user names in the tweet
  has_tag = '#[A-Za-z0-9]+' # Pattern for matching the hasgtags in the tweer
  url = 'https?:\/\/[A-Za-z0-9\.\/\-]+' # Pattern for matching the URLs in the tweet
  #emoji_pattern = r'/[x{1F600}-x{1F64F}]/u'
  
  x = text.lower()  # Lower casing all the characters
  x = re.sub(user_name, '', x) # Replace the username with an empty characrer
  x = re.sub(has_tag,'',x) # Replace the hashtags with an empty characrer
  x = re.sub(url, '', x) # Replace the URLs with an empty characrer
  #x = re.sub(emoji_pattern, '', x)
  x = x.translate(str.maketrans('', '', string.punctuation)) # Replace all the characters except alphabets and digits from the tweet

  return x

In [9]:
train_df['text'] = train_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe
valid_df['text'] = valid_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe
test_df['text'] = test_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe

In [10]:
np.save('NumpyFiles\Kan\Fulldata\Kan_Train_Full_data.npy',list(train_df['text']))
np.save('NumpyFiles\Kan\Fulldata\Kan_Train_Full_labels.npy',list(train_df['category']))
np.save('NumpyFiles\Kan\Fulldata\Kan_Val_Full_data.npy',list(valid_df['text']))
np.save('NumpyFiles\Kan\Fulldata\Kan_Val_Full_labels.npy',list(valid_df['category']))
np.save('NumpyFiles\Kan\Fulldata\Kan_Test_Full_data.npy',list(valid_df['text']))

In [11]:
temp1 = np.load('NumpyFiles\Kan\Fulldata\Kan_Train_Full_data.npy')
temp2 = np.load('NumpyFiles\Kan\Fulldata\Kan_Train_Full_labels.npy')
temp3 = np.load('NumpyFiles\Kan\Fulldata\Kan_Val_Full_data.npy')
temp4 = np.load('NumpyFiles\Kan\Fulldata\Kan_Val_Full_labels.npy')
temp5 = np.load('NumpyFiles\Kan\Fulldata\Kan_Test_Full_data.npy')

In [12]:
temp1.shape, temp2.shape, temp3.shape, temp4.shape

((6212,), (6212,), (691,), (691,))

# Dividing train and valid separatly:

### Train data

In [13]:
df_temp2 = train_df.sort_values('category')
df_temp2

,text,category
6211,26 m views,Mixed feelings
4479,yalru hands up madi avane srimannarayana,Mixed feelings
4476,h ahamad bava ಅಣ್ಣ ಒಂದೆರಡು ಪದ ಬಿಟ್ಟು ಇನ್ನೂ ಉಳ...,Mixed feelings
547,avane shriman narayana,Mixed feelings
4475,bro nivu adthiro game yavudu,Mixed feelings
...,...,...
4082,vishnuvardhana agreed,unknown state
4731,sir enthavru jaatre dialogue,unknown state
315,,unknown state
1838,ಎಂಥ lines,unknown state


In [14]:
train = Counter(train_df['category'])
train_sort = dict(sorted(train.items(), key=lambda item: item[0]))

print('Traindf: \n',train_sort,'\n')
temp = 0
for key in train_sort:
    print(key,'starts from: ', temp)
    temp = temp + int(train_sort[key])
print(key,'ends at: ', temp)

Traindf: 
 {'Mixed feelings': 574, 'Negative': 1188, 'Positive': 2823, 'not-Kannada': 916, 'unknown state': 711} 

Mixed feelings starts from:  0
Negative starts from:  574
Positive starts from:  1762
not-Kannada starts from:  4585
unknown state starts from:  5501
unknown state ends at:  6212


In [15]:
#print(df_temp2['category'].tolist())

In [16]:
count2 = 550
train_Mixed_feelings = df_temp2[:count2]
train_Negative = df_temp2[580:580+count2]
train_Positive=df_temp2[1770:1770+count2]
train_not_kanada = df_temp2[4590:4590+count2]
train_unknown_state = df_temp2[5505:5505+count2]

In [17]:
print('total:',count2*5)
len(train_Mixed_feelings), len(train_Negative), len(train_Positive), len(train_not_kanada), len(train_unknown_state)

total: 2750


(550, 550, 550, 550, 550)

In [18]:
Train_frames = [train_Mixed_feelings, train_Negative, train_Positive, train_not_kanada, train_unknown_state ]
Train_results = pd.concat(Train_frames)
Train_results.shape

(2750, 2)

In [19]:
# Train_results

In [20]:
Counter(Train_results['category'])

Counter({'Mixed feelings': 550,
         'Negative': 550,
         'Positive': 550,
         'not-Kannada': 550,
         'unknown state': 550})

In [21]:
Sub_Train = Train_results.sample(frac = 1)

In [22]:
Sub_Train_list = Sub_Train['text'].tolist()
Sub_Train_labels_list = Sub_Train['category'].tolist()

### Val Data

In [23]:
df_temp = valid_df.sort_values('category')
df_temp.shape

(691, 2)

In [24]:
val = Counter(valid_df['category'])
val_sort = dict(sorted(val.items(), key=lambda item: item[0]))

print('\nvaliddf:\n',val_sort,'\n')
temp = 0
for key in val_sort:
    print(key,'starts from: ', temp)
    temp = temp + int(val_sort[key])

print(key,'ends at: ', temp)


validdf:
 {'Mixed feelings': 52, 'Negative': 139, 'Positive': 321, 'not-Kannada': 110, 'unknown state': 69} 

Mixed feelings starts from:  0
Negative starts from:  52
Positive starts from:  191
not-Kannada starts from:  512
unknown state starts from:  622
unknown state ends at:  691


In [25]:
count = 50
val_Mixed_feelings = df_temp[:count]
val_Negative = df_temp[55:55+count]
val_Positive=df_temp[192:192+count]
val_not_Kannada = df_temp[515:515+count]
val_unknown_state = df_temp[625:625+count]

In [26]:
print('total:',count*5)
len(val_Mixed_feelings), len(val_Negative), len(val_Positive), len(val_not_Kannada), len(val_unknown_state)

total: 250


(50, 50, 50, 50, 50)

In [27]:
val_frames = [val_Mixed_feelings, val_Negative, val_Positive, val_not_Kannada, val_unknown_state ]

val_result = pd.concat(val_frames) #concat

In [28]:
Counter(val_result['category'])

Counter({'Mixed feelings': 50,
         'Negative': 50,
         'Positive': 50,
         'not-Kannada': 50,
         'unknown state': 50})

In [29]:
Sub_val = val_result.sample(frac = 1) #shuffle

In [30]:
Sub_val_list = Sub_val['text'].tolist()
Sub_val_labels_list = Sub_val['category'].tolist()

# Saving Choosen data into numpy files

In [31]:
#"C:\Users\phvpa\Desktop\Experimants\DataSet\NumpyFiles\Kan\Split2750and250"
Kan_train = np.array(Sub_Train_list)
np.save('NumpyFiles\Kan\Split2750and250\Kan_Train_data_2750.npy',Kan_train)
Kan_train_label = np.array(Sub_Train_labels_list)
np.save('NumpyFiles\Kan\Split2750and250\Kan_Train_label_2750.npy',Kan_train_label)

In [32]:
temp21 =np.load('NumpyFiles\Kan\Split2750and250\Kan_Train_data_2750.npy')
print(temp21.shape)
temp22 =np.load('NumpyFiles\Kan\Split2750and250\Kan_Train_label_2750.npy')
print(temp22.shape)

(2750,)
(2750,)


In [33]:
Kan_val = np.array(Sub_val_list)
np.save("NumpyFiles\Kan\Split2750and250\Kan_Val_data_250.npy",Kan_val)
Kan_val_label = np.array(Sub_val_labels_list)
np.save("NumpyFiles\Kan\Split2750and250\Kan_val_label_250.npy",Kan_val_label)

In [34]:
temp11 =np.load('NumpyFiles\Kan\Split2750and250\Kan_Val_data_250.npy')
print(temp11.shape)
temp12 =np.load('NumpyFiles\Kan\Split2750and250\Kan_val_label_250.npy')
print(temp12.shape)

(250,)
(250,)


In [35]:
Traindf = pd.DataFrame({'Data' : temp21,'label' : temp22}, columns=['Data', 'label'])
Traindf

,Data,label
0,nama kannada channel dinesh ss स्सुप्पेर,not-Kannada
1,ramesh snakannanavar nlar,unknown state
2,prakash kittoyta,Negative
3,hands up super songs,not-Kannada
4,bai rakshith avri nan msg nodudri yargu agllil...,Negative
...,...,...
2745,2k dislikes rashmika maandanna fanss,Negative
2746,vitthal ಲುಚ್ಛನ್ ಫ್ಯಾನ್ಸ್ ಉರ್ಕೋತಾವ್ರೆ d boss ಬ...,Negative
2747,dana alla ok,Negative
2748,howdu gurunadre climax papa,unknown state


In [36]:
Testdf = pd.DataFrame({'Data' : temp11,'label' : temp12}, columns=['Data', 'label'])
Testdf

,Data,label
0,kannada industry ali estu dodda dodda nayakaru...,Mixed feelings
1,super song,Positive
2,rip tik tok,not-Kannada
3,mast hadu super,Positive
4,guru ninu super,Positive
...,...,...
245,idu charitre srushtiso avataranasnnyar yaru mo...,Mixed feelings
246,tejashands up ande kai ettu annadukka agutta,Mixed feelings
247,vn nim akkan keytini bosdi magna nim avvagu h...,Negative
248,nivu heliddu ondu ondu maathu sathyanee,Positive
